In [1]:
!pip install textarena smithery anthropic httpx mcp --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 14.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: textarena
    Found existing installation: TextArena 0.5.6
    Uninstalling TextArena-0.5.6:
      Successfully uninstalled TextArena-0.5.6

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import os
os.environ["ANTHROPIC_API_KEY"] = "sk..."
os.environ["E2B_API_KEY"] = "e2b..."

In [3]:
from textarena.core import Agent
import textarena as ta
import asyncio
from typing import Optional

STANDARD_GAME_PROMPT = "You are a competitive game player. Make sure you read the game instructions carefully, and always follow the required format."

class AsyncAnthropicAgent(Agent):
    """Agent class using the Anthropic Claude API to generate responses asynchronously."""
    def __init__(self, model_name: str, system_prompt: Optional[str] = STANDARD_GAME_PROMPT, max_tokens: int = 1000, temperature: float = 0.9, verbose: bool = False):
        """
        Initialize the Anthropic agent.

        Args:
            model_name (str): The name of the Claude model (e.g., "claude-3-5-sonnet-20241022").
            system_prompt (Optional[str]): The system prompt to use (default: STANDARD_GAME_PROMPT).
            max_tokens (int): The maximum number of tokens to generate.
            temperature (float): The temperature for randomness in response generation.
            verbose (bool): If True, additional debug info will be printed.
        """
        super().__init__()
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.verbose = verbose
        
        try:
            import anthropic
        except ImportError:
            raise ImportError(
                "Anthropic package is required for AsyncAnthropicAgent. "
                "Install it with: pip install anthropic"
            )
            
        self.client = anthropic.AsyncAnthropic()
    
    async def _make_request(self, observation: str) -> str:
        """Make a single API request to Anthropic and return the generated message."""
        response = await self.client.messages.create(
            model=self.model_name,
            max_tokens=self.max_tokens,
            temperature=self.temperature,
            system=self.system_prompt,
            messages=[
                {"role": "user", "content": [{"type": "text", "text": observation}]}
            ]
        )
        
        return response.content[0].text.strip()
    
    async def _retry_request(self, observation: str, retries: int = 3, delay: int = 5) -> str:
        """
        Attempt to make an API request with retries.

        Args:
            observation (str): The input to process.
            retries (int): The number of attempts to try.
            delay (int): Seconds to wait between attempts.

        Raises:
            Exception: The last exception caught if all retries fail.
        """
        last_exception = None
        for attempt in range(1, retries + 1):
            try:
                response = await self._make_request(observation)
                if self.verbose:
                    print(f"\nObservation: {observation}\nResponse: {response}")
                return response
            except Exception as e:
                last_exception = e
                print(f"Attempt {attempt} failed with error: {e}")
                if attempt < retries:
                    await asyncio.sleep(delay)
        raise last_exception
    
    async def __call__(self, observation: str) -> str:
        """
        Process the observation using the Anthropic API and return the generated response.
        
        Args:
            observation (str): The input string to process.
        
        Returns:
            str: The generated response.
        """
        if not isinstance(observation, str):
            raise ValueError(f"Observation must be a string. Received type: {type(observation)}")
        return await self._retry_request(observation)

In [5]:
import textarena as ta
import smithery
import mcp
import os
import json


class MCPAgent(AsyncAnthropicAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.url = smithery.create_smithery_url(
            "wss://server.smithery.ai/e2b/ws", {"e2bApiKey": os.environ["E2B_API_KEY"]}
        )

    async def _make_request(self, observation: str) -> str:
        """Make a single API request to Anthropic and return the generated message."""
        async with smithery.websocket_client(self.url) as streams:
            async with mcp.client.session.ClientSession(*streams) as session:

                try:
                    tools_result = await session.list_tools()
                    tools = tools_result.model_dump()["tools"]

                    tools = [
                        {"input_schema": tool.pop("inputSchema"), **tool}
                        for tool in tools
                        if "inputSchema" in tool
                    ]

                    print("Available tools:", tools)

                    final_response_text = ""
                    is_tool_call_pending = True
                    messages = [
                        {
                            "role": "user",
                            "content": [{"type": "text", "text": observation}],
                        }
                    ]

                    # Loop to handle multiple tool calls in a conversation
                    while is_tool_call_pending:
                        response = await self.client.messages.create(
                            model=self.model_name,
                            max_tokens=self.max_tokens,
                            temperature=self.temperature,
                            system=self.system_prompt,
                            messages=messages,
                            tools=tools,
                        )

                        print("Response:", response)

                        # Check if there's a tool_use in the response
                        is_tool_call_pending = False
                        for content_block in response.content:
                            if content_block.type == "tool_use":
                                is_tool_call_pending = True

                                tool_name = content_block.name
                                tool_input = content_block.input
                                tool_id = content_block.id

                                print(f"Tool called: {tool_name}")
                                print(f"Tool input: {json.dumps(tool_input, indent=2)}")

                                # Execute the tool using MCP session
                                try:
                                    tool_result = await session.call_tool(
                                        tool_name, tool_input
                                    )

                                    # Convert tool result to string format for Anthropic
                                    # The content must be a string, not an object
                                    tool_result_dict = tool_result.model_dump()
                                except Exception as e:
                                    if "MCP error" in str(e):
                                        tool_result_dict = {"error": str(e)}

                                result_str = json.dumps(tool_result_dict)
                                print(f"Tool result: {result_str}")

                                # Add tool call and result to messages
                                messages.append(
                                    {
                                        "role": "assistant",
                                        "content": [content_block.model_dump()],
                                    }
                                )

                                # Add tool response to messages - content must be a string
                                messages.append(
                                    {
                                        "role": "user",
                                        "content": [
                                            {
                                                "type": "tool_result",
                                                "tool_use_id": tool_id,
                                                "content": result_str,  # Now it's a string
                                            }
                                        ],
                                    }
                                )
                            elif content_block.type == "text":
                                # Accumulate text responses
                                final_response_text += content_block.text

                        # If no tool calls were made, we use the text response
                        if not is_tool_call_pending and not final_response_text:
                            final_response_text = response.content[0].text

                except Exception as e:

                    print(f"Error: {e}")
                    raise e

            return final_response_text.strip()

In [6]:
import textarena as ta

# Initialize agents
agents = {
    0: MCPAgent(model_name="claude-3-7-sonnet-20250219"),
    1: AsyncAnthropicAgent(model_name="claude-3-5-haiku-20241022"),
}

# Initialize environment from subset and wrap it
env = ta.make(env_id="SimpleNegotiation-v0")
env = ta.wrappers.LLMObservationWrapper(env=env)
env = ta.wrappers.SimpleRenderWrapper(
    env=env,
    player_names={0: "sonnet", 1: "haiku"},
)

env.reset(num_players=len(agents))
done = False
while not done:
    player_id, observation = env.get_observation()
    action = await agents[player_id](observation)
    done, info = env.step(action=action)
    print("step complete")
rewards = env.close()
print(rewards)

Available tools: [{'input_schema': {'type': 'object', 'properties': {'code': {'type': 'string'}}, 'required': ['code'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, 'name': 'run_code', 'description': 'Run python code in a secure sandbox by E2B. Using the Jupyter Notebook syntax.'}]
Response: Message(id='msg_011pxhZEfJJX7BQG1KmS67Bs', content=[TextBlock(citations=None, text="I'll play the Negotiation Game as Player 0. Let me first analyze my current resources and their values to determine good trading strategies.", type='text'), ToolUseBlock(id='toolu_01XQCKNCQ1wkDSi6d2KyjX7J', input={'code': '# Analyze my current resources and their values\nresources = {\n    "Wheat": {"qty": 5, "value": 5, "value_per_unit": 5/5},\n    "Wood": {"qty": 17, "value": 12, "value_per_unit": 12/17},\n    "Sheep": {"qty": 18, "value": 12, "value_per_unit": 12/18},\n    "Brick": {"qty": 14, "value": 28, "value_per_unit": 28/14},\n    "Ore": {"qty": 14, "value": 39, "val

╭────────────────────────────────────────────────── Game State ───────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─────────────── Basic Information ───────────────╮ ╭─────────────────── player_resources ────────────────────╮ │
│ │                                                 │ │                                                         │ │
│ │                ╷                                │ │          ╷       ╷     ╷       ╷       ╷                │ │
│ │   Key          │ Value                          │ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood           │ │
│ │  ══════════════╪═══════                         │ │  ════════╪═══════╪═════╪═══════╪═══════╪══════          │ │
│ │   current_turn │ 1                              │ │   sonnet │ 14    │ 14  │ 18    │ 5     │ 17             │ │
│ │   max_turns    │ 10                             │ │   haiku  │ 22    │ 20  │ 21    │ 6     │ 13             │ │
│ │                ╵                                │ │          ╵       ╵     ╵       ╵       ╵                │ │
│ │                                                 │ │                                                         │ │
│ ╰─────────────────────────────────────────────────╯ ╰─────────────────────────────────────────────────────────╯ │
│ ╭──────────────── inventory_value ────────────────╮                                                             │
│ │                                                 │                                                             │
│ │          ╷        ╷         ╷                   │                                                             │
│ │   Player │ change │ current │ initial           │                                                             │
│ │  ════════╪════════╪═════════╪═════════          │                                                             │
│ │   sonnet │ 0      │ 1383    │ 1383              │                                                             │
│ │   haiku  │ 0      │ 1712    │ 1712              │                                                             │
│ │          ╵        ╵         ╵                   │                                                             │
│ │                                                 │                                                             │
│ ╰─────────────────────────────────────────────────╯                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────── Game State ────────────────────────╮╭──────────────────── Game Log ─────────────────────╮
│                                                            ││                                                   │
│ ╭────────────────── Basic Information ───────────────────╮ ││ [GAME]: Game started.                             │
│ │                                                        │ ││ sonnet: I'll play the Negotiation Game as Player  │
│ │                ╷                                       │ ││ 0. Let me first analyze my current resources and  │
│ │   Key          │ Value                                 │ ││ their values to                                   │
│ │  ══════════════╪═══════                                │ ││ determine good trading strategies.Hello! I'm      │
│ │   current_turn │ 1                                     │ ││ Player 0 in the Negotiation Game. Looking at my   │
│ │   max_turns    │ 10                                    │ ││ resources, I can see                              │
│ │                ╵                                       │ ││ that Ore and Brick have the highest value per     │
│ │                                                        │ ││ unit, while Sheep and Wood have the lowest. I     │
│ ╰────────────────────────────────────────────────────────╯ ││ should try to trade my                            │
│ ╭─────────────────── player_resources ───────────────────╮ ││ lower-value resources for higher-value ones.      │
│ │                                                        │ ││                                                   │
│ │          ╷       ╷     ╷       ╷       ╷               │ ││ I notice I have quite a lot of Sheep (18) and     │
│ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood          │ ││ Wood (17), which are my lowest value-per-unit     │
│ │  ════════╪═══════╪═════╪═══════╪═══════╪══════         │ ││ resources. I'd like to                            │
│ │   sonnet │ 14    │ 14  │ 18    │ 5     │ 17            │ ││ propose a trade that would be beneficial for both │
│ │   haiku  │ 22    │ 20  │ 21    │ 6     │ 13            │ ││ of us, focusing on exchanging some of my abundant │
│ │          ╵       ╵     ╵       ╵       ╵               │ ││ low-value                                         │
│ │                                                        │ ││ resources for higher-value ones.                  │
│ ╰────────────────────────────────────────────────────────╯ ││                                                   │
│ ╭─────────────────── inventory_value ────────────────────╮ ││ [Offer: 5 Sheep, 4 Wood -> 2 Ore]                 │
│ │                                                        │ ││                                                   │
│ │          ╷        ╷         ╷                          │ ││ This would be a fair trade as I'm offering        │
│ │   Player │ change │ current │ initial                  │ ││ resources that I have in abundance. Let me know   │
│ │  ════════╪════════╪═════════╪═════════                 │ ││ what you think or if you                          │
│ │   sonnet │ 0      │ 1383    │ 1383                     │ ││ have a counter-offer!                             │
│ │   haiku  │ 0      │ 1712    │ 1712                     │ ││ [GAME]: Player 0 made the following offer to      │
│ │          ╵        ╵         ╵                          │ ││ Player 1: Offered items: 5 Sheep, 4 Wood ->       │
│ │                                                        │ ││ Requested items: 2 Ore                            │
│ ╰────────────────────────────────────────────────────────╯ ││                                                   │
│                                                            ││                                                   │
│                                                       

step complete


╭────────────────────────────────────────────────── Game State ───────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─────────────── Basic Information ───────────────╮ ╭─────────────────── player_resources ────────────────────╮ │
│ │                                                 │ │                                                         │ │
│ │                ╷                                │ │          ╷       ╷     ╷       ╷       ╷                │ │
│ │   Key          │ Value                          │ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood           │ │
│ │  ══════════════╪═══════                         │ │  ════════╪═══════╪═════╪═══════╪═══════╪══════          │ │
│ │   current_turn │ 2                              │ │   sonnet │ 14    │ 14  │ 18    │ 5     │ 17             │ │
│ │   max_turns    │ 10                             │ │   haiku  │ 22    │ 20  │ 21    │ 6     │ 13             │ │
│ │                ╵                                │ │          ╵       ╵     ╵       ╵       ╵                │ │
│ │                                                 │ │                                                         │ │
│ ╰─────────────────────────────────────────────────╯ ╰─────────────────────────────────────────────────────────╯ │
│ ╭──────────────── inventory_value ────────────────╮                                                             │
│ │                                                 │                                                             │
│ │          ╷        ╷         ╷                   │                                                             │
│ │   Player │ change │ current │ initial           │                                                             │
│ │  ════════╪════════╪═════════╪═════════          │                                                             │
│ │   sonnet │ 0      │ 1383    │ 1383              │                                                             │
│ │   haiku  │ 0      │ 1712    │ 1712              │                                                             │
│ │          ╵        ╵         ╵                   │                                                             │
│ │                                                 │                                                             │
│ ╰─────────────────────────────────────────────────╯                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────── Game State ────────────────────────╮╭──────────────────── Game Log ─────────────────────╮
│                                                            ││                                                   │
│ ╭────────────────── Basic Information ───────────────────╮ ││ Would you like me to continue playing as Player 1 │
│ │                                                        │ ││ for the next turn?                                │
│ │                ╷                                       │ ││ sonnet: I'll help you analyze your updated        │
│ │   Key          │ Value                                 │ ││ resources after the trade and determine a good    │
│ │  ══════════════╪═══════                                │ ││ strategy for your next                            │
│ │   current_turn │ 9                                     │ ││ move. Let me calculate the new resource totals    │
│ │   max_turns    │ 10                                    │ ││ and values.I'll need to correct the calculation,  │
│ │                ╵                                       │ ││ as there seems to                                 │
│ │                                                        │ ││ be an issue with how I approached the resource    │
│ ╰────────────────────────────────────────────────────────╯ ││ values.I'll play the Negotiation Game as Player   │
│ ╭─────────────────── player_resources ───────────────────╮ ││ 0. Let me analyze my                              │
│ │                                                        │ ││ updated resources after the successful trade and  │
│ │          ╷       ╷     ╷       ╷       ╷               │ ││ plan my next move.                                │
│ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood          │ ││                                                   │
│ │  ════════╪═══════╪═════╪═══════╪═══════╪══════         │ ││ Looking at my current resources:                  │
│ │   sonnet │ 12    │ 9   │ 26    │ 5     │ 23            │ ││ - Wheat: Qty: 5, Value: 5.00 (1.00 per unit)      │
│ │   haiku  │ 24    │ 25  │ 13    │ 6     │ 7             │ ││ - Wood: Qty: 14, Value: 9.88 (0.71 per unit)      │
│ │          ╵       ╵     ╵       ╵       ╵               │ ││ - Sheep: Qty: 14, Value: 9.33 (0.67 per unit)     │
│ │                                                        │ ││ - Brick: Qty: 14, Value: 28.00 (2.00 per unit)    │
│ ╰────────────────────────────────────────────────────────╯ ││ - Ore: Qty: 17, Value: 47.36 (2.79 per unit)      │
│ ╭─────────────────── inventory_value ────────────────────╮ ││                                                   │
│ │                                                        │ ││ Our first trade was very successful! I gained     │
│ │          ╷        ╷         ╷                          │ ││ 3.57 in total value by trading my lower-value     │
│ │   Player │ change │ current │ initial                  │ ││ resources (Wood and                               │
│ │  ════════╪════════╪═════════╪═════════                 │ ││ Sheep) for higher-value Ore. I still have plenty  │
│ │   sonnet │ -83    │ 1300    │ 1383                     │ ││ of Wood and Sheep, which are my lowest            │
│ │   haiku  │ 74     │ 1786    │ 1712                     │ ││ value-per-unit resources,                         │
│ │          ╵        ╵         ╵                          │ ││ so I'll continue with a similar strategy.         │
│ │                                                        │ ││                                                   │
│ ╰────────────────────────────────────────────────────────╯ ││ I notice that Brick has the second-highest value  │
│                                                            ││ per unit after Ore. I'd like to propose another   │
│                                                       

step complete


╭────────────────────────────────────────────────── Game State ───────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─────────────── Basic Information ───────────────╮ ╭─────────────────── player_resources ────────────────────╮ │
│ │                                                 │ │                                                         │ │
│ │                ╷                                │ │          ╷       ╷     ╷       ╷       ╷                │ │
│ │   Key          │ Value                          │ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood           │ │
│ │  ══════════════╪═══════                         │ │  ════════╪═══════╪═════╪═══════╪═══════╪══════          │ │
│ │   current_turn │ 9                              │ │   sonnet │ 12    │ 9   │ 26    │ 5     │ 23             │ │
│ │   max_turns    │ 10                             │ │   haiku  │ 24    │ 25  │ 13    │ 6     │ 7              │ │
│ │                ╵                                │ │          ╵       ╵     ╵       ╵       ╵                │ │
│ │                                                 │ │                                                         │ │
│ ╰─────────────────────────────────────────────────╯ ╰─────────────────────────────────────────────────────────╯ │
│ ╭──────────────── inventory_value ────────────────╮                                                             │
│ │                                                 │                                                             │
│ │          ╷        ╷         ╷                   │                                                             │
│ │   Player │ change │ current │ initial           │                                                             │
│ │  ════════╪════════╪═════════╪═════════          │                                                             │
│ │   sonnet │ -83    │ 1300    │ 1383              │                                                             │
│ │   haiku  │ 74     │ 1786    │ 1712              │                                                             │
│ │          ╵        ╵         ╵                   │                                                             │
│ │                                                 │                                                             │
│ ╰─────────────────────────────────────────────────╯                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────── Game State ────────────────────────╮╭──────────────────── Game Log ─────────────────────╮
│                                                            ││                                                   │
│ ╭────────────────── Basic Information ───────────────────╮ ││ [GAME]: Player 0 made the following offer to      │
│ │                                                        │ ││ Player 1: Offered items: 4 Sheep, 3 Wood ->       │
│ │                ╷                                       │ ││ Requested items: 2 Brick,                         │
│ │   Key          │ Value                                 │ ││ 1 Ore                                             │
│ │  ══════════════╪═══════                                │ ││ haiku: I'll carefully analyze Player 0's latest   │
│ │   current_turn │ 9                                     │ ││ trade offer and evaluate its strategic            │
│ │   max_turns    │ 10                                    │ ││ implications:                                     │
│ │                ╵                                       │ ││                                                   │
│ │                                                        │ ││ Current Resource Status:                          │
│ ╰────────────────────────────────────────────────────────╯ ││ - [Wheat]   Qty: 6    Value: 5                    │
│ ╭─────────────────── player_resources ───────────────────╮ ││ - [Wood]    Qty: 13   Value: 9                    │
│ │                                                        │ ││ - [Sheep]   Qty: 17   Value: 13 (reduced from     │
│ │          ╷       ╷     ╷       ╷       ╷               │ ││ previous trade)                                   │
│ │   Player │ Brick │ Ore │ Sheep │ Wheat │ Wood          │ ││ - [Brick]   Qty: 22   Value: 26                   │
│ │  ════════╪═══════╪═════╪═══════╪═══════╪══════         │ ││ - [Ore]     Qty: 20   Value: 36                   │
│ │   sonnet │ 12    │ 9   │ 26    │ 5     │ 23            │ ││                                                   │
│ │   haiku  │ 24    │ 25  │ 13    │ 6     │ 7             │ ││ Analyzing the Trade Offer:                        │
│ │          ╵       ╵     ╵       ╵       ╵               │ ││ - Player 0 offers: 4 Sheep, 3 Wood                │
│ │                                                        │ ││ - Requesting: 2 Brick, 1 Ore                      │
│ ╰────────────────────────────────────────────────────────╯ ││                                                   │
│ ╭─────────────────── inventory_value ────────────────────╮ ││ Quick value calculation:                          │
│ │                                                        │ ││ - Resources offered: 4 Sheep (value ~2.68) + 3    │
│ │          ╷        ╷         ╷                          │ ││ Wood (value ~2.13) = 4.81                         │
│ │   Player │ change │ current │ initial                  │ ││ - Resources requested: 2 Brick (value ~4) + 1 Ore │
│ │  ════════╪════════╪═════════╪═════════                 │ ││ (value ~1.8) = 5.80                               │
│ │   sonnet │ -83    │ 1300    │ 1383                     │ ││                                                   │
│ │   haiku  │ 74     │ 1786    │ 1712                     │ ││ While the trade slightly favors Player 0, I see   │
│ │          ╵        ╵         ╵                          │ ││ potential strategic value. I'll counter with a    │
│ │                                                        │ ││ minor adjustment:                                 │
│ ╰────────────────────────────────────────────────────────╯ ││                                                   │
│                                                            ││ [Offer: 4 Sheep, 3 Wood -> 2 Brick, 2 Ore]        │
│                                                       

step complete
